<div class="pull-left"><left><img style="float: right;" src="http://developer.ibm.com/clouddataservices/wp-content/uploads/sites/85/2018/01/ibm-cloud-object-storage-logo-small.png" width="100" margin=50></left></div>
<div style="text-align:center">
**IBM Cloud SQL Query** is IBM's **serverless SQL** service on data in **Cloud Object Storage**. It allows to run ANSI SQL on Parquet, CSV, JSON, ORC and AVRO data sets. You can use it to run your analytic queries, and you can use it to conduct complex transformations and write the result in any desired data format, partitioning and layout. SQL Query is based on Apache Spark SQL as the query engine in the background. This means you do **not** have to provision any Apache Spark instance or service. A simple Python client (like the IBM Watson Studio Notebook) is sufficient.<br><br></div>
This notebook is meant to be a generic starter to use the SQL Query API in order to run SQL statements in a programmatic way. It uses the <a href="https://github.com/IBM-Cloud/sql-query-clients/tree/master/Python" target="_blank" rel="noopener noreferrer">ibmcloudsql</a> Python library for this purpose. The notebook also demonstrates how you can combine SQL Query with visualization libraries such as **PixieDust**. The notebook has been verified to work with Python 3.5. As mentioned above it does not require a Spark service bound to the notebook.

## Table of contents
1. [Setup libraries](#setup)<br>
2. [Configure SQL Query](#configure)<br>
    2.1 [Using the project bucket](#projectbucket)<br>
    2.2 [Setting SQL Query parameters](#parameters)<br>
3. [Your SQL](#sql)<br>
4. [Running Your SQL Statement](#run)<br>
    4.1 [Low level SQL job submission](#lowlevel)<br>
5. [List recent SQL submissions](#joblist)<br>
6. [Next steps](#next)<br>

### <a id="setup"></a> 1. Setup libraries

Run the following cell at least once in your notebook environment in order to install required packages, such as the SQL Query client library:

In [ ]:
!conda install pyarrow
!conda install sqlparse

In [ ]:
!pip install --user ibmcloudsql

In [ ]:
import ibmcloudsql
from pixiedust.display import *
import pandas as pd
targeturl=''

### <a id="configure"></a> 2. Configure SQL Query
1. You need an **API key** for an IBM cloud identity that has access to your Cloud Object Storage bucket for writing SQL results and to your SQL Query instance. To create API keys log on to the IBM Cloud console and go to <a href="https://console.bluemix.net/iam/#/apikeys" target="_blank">Manage->Security->Platform API Keys</a>, click the `Create` button, give the key a custom name and click `Create`. In the next dialog click `Show` and copy the key to your clipboard and paste it below in this notebook.
2. You need the **instance CRN** for the SQL Query instance. You can find it in the <a href="https://console.bluemix.net/dashboard/apps" target="_blank">IBM Cloud console dashboard</a>. Make sure you have `All Resources` selected as resource group. In the section `Services` you can see your instances of SQL Query and Cloud Object Storage. Select the instance of SQL Query that you want to use. In the SQL Query dashboard page that opens up you find a section titled **REST API** with a button labelled **Instance CRN**. Click the button to copy the CRN into your clipboard and paste it here into the notebook. If you don't have an SQL Query instance created yet, <a href="https://console.bluemix.net/catalog/services/sql-query" target="_blank">create one</a> first.
3. You need to specify the location on Cloud Object Storage where your **query results** should be written. This comprises three parts of information that you can find in the Cloud Object Storage UI for your instance in the IBM Cloud console. You need to provide it as a **URL** using the format `cos://<endpoint>/<bucket>/[<prefix>]`. You have the option to use the cloud object storage **bucket that is associated with your project**. In this case, execute the following section before you proceed.  
<br/>
For more background information, check out the SQL Query <a href="https://console.bluemix.net/docs/services/sql-query/getting-started.html#getting-started-tutorial" target="_blank">documentation</a>.

#### <a id="projectbucket"></a> 2.1 Using the project bucket
**Only** follow the instructions in this section when you want to write your SQL query results to the bucket that has been created for the project for which you have created this notebook. In any other case proceed directly with section **2.2**.
<br><br>
__Inserting the project token__:  
Click the `More` option in the toolbar above (the three stacked dots) and select `Insert project token`.
 * If you haven't created an access token for this project before, you will see a dialog that asks you to create one first. Follow the link to open your project settings, scroll down to `Access tokens` and click `New token`. Give the token a custom name and make sure you select `Editor` as `Access role for project`. After you created your access token you can come back to this notebook, select the empty cell below and again select `Insert project token` from the toolbar at the top.
[//]: # 
This will add a new cell at the top of your notebook with content that looks like this:
```
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='<some id>', project_access_token='<some access token>')
pc = project.project_context
```
Leave that cell content as inserted and run the cell. Then then proceed with the following cell below:

In [ ]:
cos_bucket = project.get_metadata()['entity']['storage']['properties']
targeturl="cos://" + cos_bucket['bucket_region'] + "/" + cos_bucket['bucket_name'] + "/"

#### <a id="parameters"></a> 2.2 Setting the SQL Query parameters

In [ ]:
import getpass
apikey=getpass.getpass('Enter IBM Cloud API Key (leave empty to use previous one): ') or apikey
instnacecrn=input('Enter SQL Query Instance CRN (leave empty to use previous one): ') or instnacecrn
if targeturl == '':
    targeturl=input('Enter target URL for SQL results: ')
else:
    targeturl=input('Enter target URL for SQL results (leave empty to use ' + targeturl + '): ') or targeturl
sqlClient = ibmcloudsql.SQLQuery(apikey, instnacecrn, client_info='SQL Query Starter Notebook')
sqlClient.logon()
print('\nYour SQL Query web console link:\n')
sqlClient.sql_ui_link()

### <a id="sql"></a> 3. Your SQL
To author your own SQL query, use the interactive SQL Query web console (**link above**) of your SQL Query service instance.

In [7]:
import sqlparse
from pygments import highlight
from pygments.lexers import get_lexer_by_name
from pygments.formatters import HtmlFormatter, Terminal256Formatter

sql=input('Enter your SQL statement (leave empty to use a simple sample SQL)')
if sql == '':
    sql='SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, \
         cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c \
         WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > "1998-01-01" \
         ORDER BY c.CompanyName'
if " INTO " not in sql:
    sql += ' INTO {} STORED AS CSV'.format(targeturl)
formatted_sql = sqlparse.format(sql, reindent=True, indent_tabs=True, keyword_case='upper')
lexer = get_lexer_by_name("sql", stripall=True)
formatter = Terminal256Formatter(style='tango')
result = highlight(formatted_sql, lexer, formatter)
from IPython.core.display import display, HTML
print('\nYour SQL statement is:\n')
print(result)

Enter your SQL statement (leave empty to use a simple sample SQL)

Your SQL statement is:

SELECT o.OrderID,
	c.CompanyName,
	e.FirstName,
	e.LastName
FROM cos://us-geo/SQL/orders.parquet STORED AS PARQUET o,
	cos://us-geo/SQL/employees.parquet STORED AS PARQUET e,
	cos://us-geo/SQL/customers.parquet STORED AS PARQUET c
WHERE e.EmployeeID = o.EmployeeID
		AND c.CustomerID = o.CustomerID
		AND o.ShippedDate > o.RequiredDate
		AND o.OrderDate > "1998-01-01"
ORDER BY c.CompanyName INTO cos://us-south/sqltempregional/ STORED AS CSV



### <a id="run"></a> 4. Running Your SQL Statement
The following cell submits the above SQL statement and waits for it to finish before printing a sample of the result set.

In [8]:
result_df = sqlClient.run_sql(sql)
if isinstance(result_df, str):
    print(result_df)

In [9]:
result_df.head(10)

,OrderID,CompanyName,FirstName,LastName
0,10924,Berglunds snabbköp,Janet,Leverling
1,11058,Blauer See Delikatessen,Anne,Dodsworth
2,10827,Bon app',Nancy,Davolio
3,11076,Bon app',Margaret,Peacock
4,11045,Bottom-Dollar Markets,Michael,Suyama
5,10970,Bólido Comidas preparadas,Anne,Dodsworth
6,11054,Cactus Comidas para llevar,Laura,Callahan
7,11008,Ernst Handel,Robert,King
8,11072,Ernst Handel,Margaret,Peacock
9,10816,Great Lakes Food Market,Margaret,Peacock


In [10]:
from pixiedust.display import *
display(result_df)

#### <a id="lowlevel"></a> 4.1 Low level SQL job submission
Let's run the same SQL again, but this time using the asynchronous submission mechanism and the status check method.

In [11]:
sqlClient.logon()
jobId = sqlClient.submit_sql(sql)
print("SQL query submitted and running in the background. jobId = " + jobId)

SQL query submitted and running in the background. jobId = 720fc241-b982-4ff9-8bfb-ec18638c625e


In [12]:
print("Job status for " + jobId + ": " + sqlClient.get_job(jobId)['status'])

Job status for 720fc241-b982-4ff9-8bfb-ec18638c625e: running


Use the `wait_for_job()` method as a blocking call until your job has finished:

In [13]:
job_status = sqlClient.wait_for_job(jobId)
print("Job " + jobId + " terminated with status: " + job_status)
if job_status == 'failed':
    details = sqlClient.get_job(jobId)
    print("Error: {}\nError Message: {}".format(details['error'], details['error_message']))

Job 720fc241-b982-4ff9-8bfb-ec18638c625e terminated with status: completed


Use the `get_result()` method to retrieve a dataframe for the SQL result set:

In [14]:
result_df = sqlClient.get_result(jobId)
print("OK, we have a dataframe for the SQL result that has been stored by SQL Query in " + sqlClient.get_job(jobId)['resultset_location'])

OK, we have a dataframe for the SQL result that has been stored by SQL Query in cos://s3.us-south.objectstorage.softlayer.net/sqltempregional/jobid=720fc241-b982-4ff9-8bfb-ec18638c625e


You can delete the result set from Cloud Object Storage using the `delete_result()` method:

In [15]:
sqlClient.delete_result(jobId)

,Deleted Object
0,jobid=720fc241-b982-4ff9-8bfb-ec18638c625e
1,jobid=720fc241-b982-4ff9-8bfb-ec18638c625e/_SU...
2,jobid=720fc241-b982-4ff9-8bfb-ec18638c625e/par...


### <a id="joblist"></a> 5. Working with your SQL Job Submission History
The following cell uses the `get_cos_summary()` method to get a statistical overview of the data in the **target location** in COS that has been used by the above queries in this notebook.

In [19]:
sqlClient.get_cos_summary(targeturl)

{'largest_object': 'jobid=778d78b9-1068-4fd4-86d8-793fc1b5b737/part-00000-43ce56b3-4e44-4841-81e1-3d1e00bf8f43.csv-attempt_20180403103903_0009_m_000000_0',
 'largest_object_size': '203.4 MB',
 'newest_object_timestamp': 'October 25, 2018, 11H:22M:50S',
 'oldest_object_timestamp': 'November 24, 2017, 09H:58M:25S',
 'smallest_object': 'categories.avro',
 'smallest_object_size': '0.0 B',
 'total_objects': 7291,
 'total_volume': '1.4 GB',
 'url': 'cos://us-south/sqltempregional/'}

The method `get_jobs()` provides you a dataframe with the **30 most recent SQL submissions** with all details. You can change the value `-1`for `display.max_colwidth` to a positive integer if you want to truncate the cell content to shrink the overall table display size.

In [20]:
pd.set_option('display.max_colwidth', -1)
job_history_df = sqlClient.get_jobs()
job_history_df.head(100)

,job_id,status,user_id,statement,resultset_location,submit_time,end_time,rows_read,rows_returned,bytes_read,error,error_message
0,fc50b138-5e8e-4963-8662-131c9febc41d,completed,torsten@de.ibm.com,"SELECT * FROM cos://s3.us-south.objectstorage.softlayer.net/sqltempregional/customer_orders/jobid=2713410f-492d-482e-9a55-f9ecf0b83987 STORED AS PARQUET WHERE ShipCountry = ""Austria"" AND ShipCity=""Graz"" INTO cos://us-south/sqltempregional/ STORED AS PARQUET",cos://s3.us-south.objectstorage.softlayer.net/sqltempregional/jobid=fc50b138-5e8e-4963-8662-131c9febc41d,2018-10-25T11:22:25.594Z,2018-10-25T11:22:52.976Z,30,30,6090,None,None
1,2713410f-492d-482e-9a55-f9ecf0b83987,completed,torsten@de.ibm.com,"SELECT OrderID, c.CustomerID CustomerID, CompanyName, ContactName, ContactTitle, Address, City, Region, PostalCode, Country, Phone, Fax EmployeeID, OrderDate, RequiredDate, ShippedDate, ShipVia, Freight, ShipName, ShipAddress, ShipCity, ShipRegion, ShipPostalCode, ShipCountry FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE c.CustomerID = o.CustomerID INTO cos://us-south/sqltempregional/customer_orders STORED AS PARQUET PARTITIONED BY (ShipCountry, ShipCity)",cos://s3.us-south.objectstorage.softlayer.net/sqltempregional/customer_orders/jobid=2713410f-492d-482e-9a55-f9ecf0b83987,2018-10-25T11:21:51.894Z,2018-10-25T11:22:05.037Z,921,830,43058,None,None
2,002f9acd-f451-4af3-8a57-0f35faf5937b,completed,torsten@de.ibm.com,"SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > ""1998-01-01"" ORDER BY c.CompanyName INTO cos://us-south/sqltempregional/ STORED AS CSV",cos://s3.us-south.objectstorage.softlayer.net/sqltempregional/jobid=002f9acd-f451-4af3-8a57-0f35faf5937b,2018-10-25T11:19:40.316Z,2018-10-25T11:20:11.880Z,1760,29,41499,None,None
3,c1d1603d-dfb2-425d-9b3e-db39adad0fb1,completed,torsten@de.ibm.com,"SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > ""1998-01-01"" ORDER BY c.CompanyName INTO cos://us-south/sqltempregional/ STORED AS CSV",cos://s3.us-south.objectstorage.softlayer.net/sqltempregional/jobid=c1d1603d-dfb2-425d-9b3e-db39adad0fb1,2018-10-25T11:19:19.954Z,2018-10-25T11:19:27.630Z,1760,29,41499,None,None
4,3b47a82d-a69d-45ce-8f8d-129ed7bb9f46,completed,torsten@de.ibm.com,SELECT * FROM cos://us-south/sqltempregional/my_job_history/ STORED AS PARQUET INTO cos://us-south/sqltempregional/ STORED AS CSV,cos://s3.us-south.objectstorage.softlayer.net/sqltempregional/jobid=3b47a82d-a69d-45ce-8f8d-129ed7bb9f46,2018-10-25T09:06:32.429Z,2018-10-25T09:06:35.331Z,30,30,9384,None,None
5,11be8f97-770a-4094-aa07-a65b6089a954,completed,torsten@de.ibm.com,"-- Example of a case expression:\nSELECT e.FirstName, e.LastName, COUNT(o.OrderID) As NumOrders,\n (CASE WHEN o.ShippedDate < o.RequiredDate\n\t\t\tTHEN 'On Time'\n\t\t\tELSE 'Late'\n END) AS Shipped\nFROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o,\n cos://us-geo/sql/employees.parquet STORED AS PARQUET e\nWHERE e.EmployeeID = o.EmployeeID\nGROUP BY e.FirstName, e.LastName,\n (CASE WHEN o.ShippedDate < o.RequiredDate\n THEN 'On Time'\n ELSE 'Late'\n END)\nORDER BY e.LastName, e.FirstName, NumOrders DESC\n\n-- This query creates a report that generates, for each employee and only if the corresponding\n-- number of orders is >0:\n-- - One record indicating the number of orders associated with that employee that shipped on time\n-- - Another

In [21]:
sqlClient.export_job_history(targeturl + "my_job_history/")

Exported 5 new jobs


In [22]:
sql = "SELECT * FROM {}my_job_history/ STORED AS PARQUET INTO {} STORED AS CSV".format(targeturl, targeturl)
sqlClient.run_sql(sql).head(100)

,index,job_id,status,user_id,statement,resultset_location,submit_time,end_time,rows_read,rows_returned,bytes_read,error,error_message
0,0,11be8f97-770a-4094-aa07-a65b6089a954,completed,torsten@de.ibm.com,"-- Example of a case expression:\nSELECT e.FirstName, e.LastName, COUNT(o.OrderID) As NumOrders,\n (CASE WHEN o.ShippedDate < o.RequiredDate\n\t\t\tTHEN 'On Time'\n\t\t\tELSE 'Late'\n END) AS Shipped\nFROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o,\n cos://us-geo/sql/employees.parquet STORED AS PARQUET e\nWHERE e.EmployeeID = o.EmployeeID\nGROUP BY e.FirstName, e.LastName,\n (CASE WHEN o.ShippedDate < o.RequiredDate\n THEN 'On Time'\n ELSE 'Late'\n END)\nORDER BY e.LastName, e.FirstName, NumOrders DESC\n\n-- This query creates a report that generates, for each employee and only if the corresponding\n-- number of orders is >0:\n-- - One record indicating the number of orders associated with that employee that shipped on time\n-- - Another record indicating the number of orders that shipped late\n-- INTO clause was automatically added based on the default target\nINTO cos://us-south/sqltempregional/ STORED AS CSV",cos://s3.us-south.objectstorage.softlayer.net/sqltempregional/jobid=11be8f97-770a-4094-aa07-a65b6089a954,2018-10-25T09:05:52.954Z,2018-10-25T09:06:22.448Z,839,18,20626,NaN,NaN
1,1,ea76d7b5-6fd0-45b3-9982-125ba637e2d8,completed,torsten@de.ibm.com,"-- Example of a case expression:\nSELECT e.FirstName, e.LastName, COUNT(o.OrderID) As NumOrders,\n (CASE WHEN o.ShippedDate < o.RequiredDate\n\t\t\tTHEN 'On Time'\n\t\t\tELSE 'Late'\n END) AS Shipped\nFROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o,\n cos://us-geo/sql/employees.parquet STORED AS PARQUET e\nWHERE e.EmployeeID = o.EmployeeID\nGROUP BY e.FirstName, e.LastName,\n (CASE WHEN o.ShippedDate < o.RequiredDate\n THEN 'On Time'\n ELSE 'Late'\n END)\nORDER BY e.LastName, e.FirstName, NumOrders DESC\n\n-- This query creates a report that generates, for each employee and only if the corresponding\n-- number of orders is >0:\n-- - One record indicating the number of orders associated with that employee that shipped on time\n-- - Another record indicating the number of orders that shipped late\n-- INTO clause was automatically added based on the default target\nINTO cos://us-south/sqltempregional/ STORED AS CSV",cos://s3.us-south.objectstorage.softlayer.net/sqltempregional/jobid=ea76d7b5-6fd0-45b3-9982-125ba637e2d8,2018-10-25T09:05:51.157Z,2018-10-25T09:05:56.846Z,839,18,20626,NaN,NaN
2,2,d8d3329b-8984-45db-93cf-11b8009fa2a1,completed,torsten@de.ibm.com,"-- Example of a case expression:\nSELECT e.FirstName, e.LastName, COUNT(o.OrderID) As NumOrders,\n (CASE WHEN o.ShippedDate < o.RequiredDate\n\t\t\tTHEN 'On Time'\n\t\t\tELSE 'Late'\n END) AS Shipped\nFROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o,\n cos://us-geo/sql/employees.parquet STORED AS PARQUET e\nWHERE e.EmployeeID = o.EmployeeID\nGROUP BY e.FirstName, e.LastName,\n (CASE WHEN o.ShippedDate < o.RequiredDate\n THEN 'On Time'\n ELSE 'Late'\n END)\nORDER BY e.LastName, e.FirstName, NumOrders DESC\n\n-- This query creates a report that generates, for each employee and only if the corresponding\n-- number of orders is >0:\n-- - One record indicating the number of orders associated with that employee that shipped on time\n-- - Another record indicating the number of orders that shipped late\n-- INTO clause was automatically added based on the default target\nINTO cos://us-south/sqltempregional/ STORED AS CSV",cos://s3.us-south.objectstorage.softlayer.net/sqltempregional/jobid=d8d3329b-8984-45db-93cf-11b8009fa2a1,2018-10-25T09:05:47.453Z,2018-10-25T09:05:52.881Z,839,18,20626,NaN,NaN
3,3,0d4476ec-b7a7-4ef5-b811-bbafa61667d5,completed,torsten@de.ibm.com,"-- Example of a case expression:\nSELECT e.FirstName, e.LastName, COUNT(o.OrderID) As NumOrders,\n (CASE WHEN o.ShippedDate < o.RequiredDate\n\t\t\tTHEN 'On Time'\n\t\t\tELSE 'Late'\n END) AS Shipped\nFROM cos://us-geo/sql/orders.parquet STORED A

### <a id="next"></a> 6. Next steps
In this notebook you learned how you can use the `ibmcloudsql` library in a Python notebook to submit SQL queries on data in IBM Cloud Object Storage and how you can interact with the query results. If you want to automate such an SQL query execution as part of your cloud solution, you can use the <a href="https://console.bluemix.net/openwhisk/" target="_blank">IBM Cloud Functions</a> framework. There is a dedicated SQL function available that lets you set up a cloud function to run SQL statements with IBM Cloud SQL Query. You can find the documentation for doing this <a href="https://hub.docker.com/r/ibmfunctions/sqlquery/" target="_blank" rel="noopener noreferrer">here</a>.

### <a id="authors"></a>Authors

**Torsten Steinbach**, Torsten is the lead architect for IBM Cloud SQL Query. Previously he has worked as IBM architect for a series of data management products and services, including DB2, PureData for Analytics and Db2 on Cloud.

<hr>
Copyright &copy; IBM Corp. 2018. This notebook and its source code are released under the terms of the MIT License.